Author: [Jonathan Giezendanner](https://www.jgiezendanner.com/)\
Created On: Nov. 16th 2023

# Order, post-process and download files from LAADS DAC

The goal of this notebook is to show how to check for file availability for a given product on LAADS DAC, request the files with post-processing, and finally download the data.

### Credentials
apikey - see here to request api key: https://ladsweb.modaps.eosdis.nasa.gov/profiles/#generate-token-modal \
email - email used to access https://ladsweb.modaps.eosdis.nasa.gov/

In [1]:
apikey = ''
email = ''

In [2]:
import datetime
from bs4 import BeautifulSoup
import time
import contextlib
from IPython.utils import io
from pathlib import Path

## Options

### Output

In [3]:
outputPath = Path('OutputTif') # folder to save the output to

### Date and RoI

In [4]:
dateOfInterest = "2022-11-20"

In [5]:
# bounding box
north = 35.35687719
south = 34.49530691
east = 11.14011447
west = 10.29190187

### Product Specification

In [6]:
product = 'VNP09'
collection = '5000'
dayNightBoth = 'D'
coordsOrTiles = 'coords'

### Post Processing

In [7]:
reformatType = "geotiff"
doMosaic = "true"
reprojectionName = "MERCAT"

In [8]:
outputPath.mkdir(exist_ok=True, parents=True)

## Get available files

In [9]:
date = datetime.datetime.strptime(dateOfInterest,"%Y-%m-%d") #for later when I need to read the date

stringDate = date.strftime("%Y-%m-%d")

commandAvailability =  "\"https://modwebsrv.modaps.eosdis.nasa.gov/axis2/services/MODAPSservices/searchForFiles?"+\
    f"products={product}&"+\
    f"collection={collection}&"+\
    f"startTime={stringDate}&"+\
    f"endTime={stringDate}&"+\
    f"north={north}&"+\
    f"east={east}&"+\
    f"south={south}&"+\
    f"west={west}&"+\
    f"dayNightBoth={dayNightBoth}&"+\
    f"coordsOrTiles={coordsOrTiles}\""

In [10]:
availabilityXML = (outputPath/'availability.xml').as_posix()
with io.capture_output() as captured:
    !wget $commandAvailability -O "$availabilityXML"

In [11]:
with open(availabilityXML, 'r') as f:
    images = ','.join(map(lambda item: item.text, BeautifulSoup(f.read(), "xml").find_all('return')))

## Order files

In [12]:
commandOrder = f"\"https://modwebsrv.modaps.eosdis.nasa.gov/axis2/services/MODAPSservices/orderFiles?"+\
             f"email={email}&"+\
             f"fileIds={images}&"+\
             f"reformatType={reformatType}&"+\
             f"doMosaic={doMosaic}&"+\
             f"reprojectionName={reprojectionName}&"+\
             f"geoSubsetNorth={north}&"+\
             f"geoSubsetEast={east}&"+\
             f"geoSubsetSouth={south}&"+\
             f"geoSubsetWest={west}\""

In [13]:
orderXML = (outputPath/'order.xml').as_posix()
with io.capture_output() as captured:
    !wget $commandOrder -O "$orderXML"

In [14]:
with open(orderXML, 'r') as f:
    orderNumber = BeautifulSoup(f.read(), "xml").find_all('return')[0].text

## Wait for order to complete

In [15]:
commandStatus = f"\"https://modwebsrv.modaps.eosdis.nasa.gov/axis2/services/MODAPSservices/getOrderStatus?"+\
             f"orderId={orderNumber}\""

In [16]:
statusXML = (outputPath/'status.xml').as_posix()
status = ''
while status != 'Available':
    time.sleep(10)
    with io.capture_output() as captured:
        !wget $commandStatus -O "$statusXML"
    with open(statusXML, 'r') as f:
        status = BeautifulSoup(f.read(), "xml").find_all('return')[0].text
    print("                            ", end='\r')
    print(status, end='\r')

## Download Files

In [17]:
outputPathString = outputPath.as_posix()
archiveURL = f"\"https://ladsweb.modaps.eosdis.nasa.gov/archive/orders/{orderNumber}/\""
access = f"\"Authorization: Bearer {apikey}\""
with io.capture_output() as captured:
    !wget -e robots=off -m -np -R .html,.tmp -nH --cut-dirs=3 $archiveURL --header $access -P $outputPath

## Release Order

In [18]:
commandRelease = f"\"https://modwebsrv.modaps.eosdis.nasa.gov/axis2/services/MODAPSservices/releaseOrder?"+\
             f"email={email}&"+\
             f"orderId={orderNumber}\""

In [19]:
releaseXML = (outputPath/'release.xml').as_posix()
with io.capture_output() as captured:
    !wget $commandRelease -O "$releaseXML"